create empty mne-python raw object

In [1]:
import numpy as np
import mne

# Create a simple RawArray
sfreq = 250  # Sampling frequency
ch_names = [f'EEG{d}' for d in range(1,75)]
ch_types = ["eeg"] * 74
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

data = np.random.randn(74, 747750)  # 2 channels, 1000 samples
raw = mne.io.RawArray(data, info)

print(isinstance(raw, mne.io.Raw))  # True
print(type(raw))  # <class 'mne.io.array.array.RawArray'>

Creating RawArray with float64 data, n_channels=74, n_times=747750
    Range : 0 ... 747749 =      0.000 ...  2990.996 secs
Ready.
False
<class 'mne.io.array.array.RawArray'>


braindecode call __getitem__ of mne.base.Raw, which then calls _getitem which calls _read_segment of BaseRaw. mne uses _read_segment to read a specific range of the file. We want to test whether S3 file via fsspec can be integrated
It calls _read_segments_file of the BaseRaw class. Any subclass must implement this method. EEGLAB calls fiff reader function: mne/_fiff/utils.py#L200

In [16]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('..')
from eegdash.data_utils import RawEEGDash
from eegdash.signalstore_data_utils import SignalstoreOpenneuro

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
signalstore = SignalstoreOpenneuro(
    is_public=False,
    local_filesystem=False,
)

Pinged your deployment. You successfully connected to MongoDB!


In [54]:
record = signalstore.find({'dataset': 'ds002718', 'subject': '002'})[0]
sfreq = record['sampling_frequency']
nchans = record['nchans']
ntimes = record['ntimes']
ch_names = record['channel_names']
ch_types = record['channel_types']
s3_path = signalstore.get_s3path(record)
print(s3_path)


Found 1 records
s3://openneuro.org/ds002718/sub-002/eeg/sub-002_task-FaceRecognition_eeg.set


In [52]:
eegdash = RawEEGDash(s3_path, {'sfreq': sfreq, 'nchans': nchans, 'n_times': ntimes, 'ch_types': ch_types, 'ch_names': ch_names}, preload=False)

ntimes 747750


integrate with braindecode

In [55]:
from braindecode.datasets import BaseDataset, BaseConcatDataset
eegdash_braindecode = BaseConcatDataset([BaseDataset(eegdash)])

In [56]:
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows)
windows_ds = create_fixed_length_windows(eegdash_braindecode, start_offset_samples=0, stop_offset_samples=None,
        window_size_samples=1000,
        window_stride_samples=1000, drop_last_window=True,
        preload=False)

In [57]:
windows_ds[0][0].shape

(74, 1000)